# FoV Optimization Result Check (fixed mode)

Use this notebook **after** running tuning and then `tune --mode fixed`.

Set `VIEW` to one of `top`, `diagonal`, or `bottom`, then run all cells.

This notebook gives you:
- sanity checks for required result files
- visibility progress per iteration
- TE / RE plots and summary
- parameter traces per iteration (`alpha`, `ks`, step bounds)
- parameter snapshot table at selected iterations
- per-pose update heatmap (`iteration x pose`)
- clean interactive 3D quiver animation with slider + play/pause


In [ ]:
import json
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

try:
    import yaml
except Exception:
    yaml = None

try:
    import plotly.graph_objects as go
except Exception:
    go = None
    print('Plotly is not installed. Install with: pip install plotly')

plt.rcParams['figure.figsize'] = (11, 4)
plt.rcParams['axes.grid'] = True


In [ ]:
TRACE_ROOT = Path('/home/shekoufeh/fov_ws/my_FIF-perception-aware-planning/act_map_exp/trace')

# Select exactly one view.
VIEW = 'top'         # 'top' | 'diagonal' | 'bottom'
VARIANT = 'none'     # e.g. 'none', 'gp_det', 'gp_trace', 'quad_det', ...
USE_PATH_YAW = True  # True -> optimized_path_yaw, False -> optimized

# Optional direct path override. If set, VIEW/VARIANT/USE_PATH_YAW are ignored.
RUN_ROOT_OVERRIDE = None

# Optional overrides for this notebook run.
PARAM_OVERRIDE = {
    # 'ks': 65.0,
    # 'fov_schedule': '90,45,15',
    # 'base_step_scale': 1.08,
    # 'min_step_deg': 0.49,
    # 'max_step_deg': 1.78,
    # 'traj_jac_step': 0.29,
}

MAX_POINTS = 4000      # None = use all points from trajectory_pointcloud.csv
POINT_STRIDE_3D = 3    # 1 = use all loaded points in 3D view
MAX_ANIM_FRAMES = 150  # downsample animation frames when iterations are many
PROBE_ITERS = [0, 1, 5, 10, -1]

# Defaults from trajectory_optimizer_copy.h
DEFAULTS = {
    'ks': 15.0,
    'fov_schedule': '180,90,60,15',
    'base_step_scale': 1.0,
    'min_step_deg': 0.25,
    'max_step_deg': 5.0,
    'traj_jac_step': 0.5,
    'min_step_decay_strength': 1.0,
    'max_step_decay_strength': 0.5,
}


In [ ]:
def parse_schedule(v):
    if v is None:
        return None
    if isinstance(v, (list, tuple)):
        out = []
        for x in v:
            try:
                out.append(float(x))
            except Exception:
                pass
        return out or None
    if isinstance(v, str):
        vals = [s.strip() for s in v.split(',') if s.strip()]
        out = []
        for s in vals:
            try:
                out.append(float(s))
            except Exception:
                pass
        return out or None
    return None


def parse_best_params(trace_root: Path):
    cands = [
        trace_root / 'optuna_best_params.yaml',
        trace_root / 'optuna_best_param.yaml',
    ]
    best_yaml = next((p for p in cands if p.exists()), None)
    fixed = {}
    if best_yaml is not None and yaml is not None:
        data = yaml.safe_load(best_yaml.read_text(encoding='utf-8')) or {}
        if isinstance(data, dict):
            fp = data.get('fixed_params', {})
            if isinstance(fp, dict):
                fixed = fp
    return best_yaml, fixed


def resolve_run_root(trace_root: Path, view: str, variant: str, use_path_yaw: bool, override=None):
    if override is not None:
        return Path(override)
    suffix = 'optimized_path_yaw' if use_path_yaw else 'optimized'
    return trace_root / view / f'{view}_{variant}' / suffix


def resolve_quiver_file(root: Path):
    cands = [
        root / 'quivers_path_yaw.txt',
        root / 'quivers.txt',
        root / 'initial_quivers_path_yaw.txt',
        root / 'initial_quivers.txt',
    ]
    for c in cands:
        if c.exists():
            return c
    raise FileNotFoundError(f'No quiver file found under {root}')


def resolve_pose_errors_file(root: Path):
    cands = [
        root / 'pose_errors_path_yaw.txt',
        root / 'pose_errors.txt',
    ]
    for c in cands:
        if c.exists():
            return c
    return None


def parse_quiver_iterations(path: Path):
    blocks, cur = [], []
    with path.open('r', encoding='utf-8') as f:
        for raw in f:
            line = raw.strip()
            if not line:
                if cur:
                    blocks.append(np.asarray(cur, dtype=float))
                    cur = []
                continue
            parts = [p.strip() for p in line.split(',') if p.strip()]
            if len(parts) >= 6:
                cur.append([float(v) for v in parts[:6]])
    if cur:
        blocks.append(np.asarray(cur, dtype=float))

    if not blocks:
        raise RuntimeError(f'No iteration blocks found in {path}')

    min_n_pose = min(len(b) for b in blocks)
    blocks = [b[:min_n_pose] for b in blocks]
    arr = np.stack(blocks, axis=0)  # [iter, pose, 6]
    return arr


def load_points(path: Path, max_points=4000, seed=7):
    pts = np.loadtxt(path, delimiter=',', usecols=(0, 1, 2), dtype=float)
    if pts.ndim == 1:
        pts = pts[None, :]
    if max_points is not None and pts.shape[0] > max_points:
        rng = np.random.default_rng(seed)
        idx = rng.choice(pts.shape[0], size=max_points, replace=False)
        pts = pts[idx]
    return pts


def parse_pose_errors(path: Path):
    te_vals, re_vals = [], []
    if path is None or not path.exists():
        return te_vals, re_vals
    with path.open('r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i == 0:
                continue
            s = line.strip().split()
            if len(s) < 3:
                continue
            if s[1] != 'nan':
                te_vals.append(float(s[1]))
            if s[2] != 'nan':
                re_vals.append(float(s[2]))
    return te_vals, re_vals


def normalize(v, eps=1e-9):
    n = np.linalg.norm(v, axis=-1, keepdims=True)
    return v / np.maximum(n, eps)


def alpha_for_iter(it, n_iter, schedule_deg):
    if schedule_deg:
        stages = len(schedule_deg)
        stage_len = max(1, n_iter // stages)
        stage_idx = min(it // stage_len, stages - 1)
        return float(schedule_deg[stage_idx])

    ratio = it / max(1, n_iter)
    if ratio < 0.05:
        return 179.0
    if ratio < 0.10:
        return 90.0
    if ratio < 0.15:
        return 60.0
    return 15.0


def build_iteration_param_trace(n_iter, params):
    ks = float(params.get('ks', DEFAULTS['ks']))
    fov_schedule = parse_schedule(params.get('fov_schedule')) or parse_schedule(DEFAULTS['fov_schedule'])
    base_step_scale = float(params.get('base_step_scale', DEFAULTS['base_step_scale']))
    min_step_deg = float(params.get('min_step_deg', DEFAULTS['min_step_deg']))
    max_step_deg = float(params.get('max_step_deg', DEFAULTS['max_step_deg']))
    traj_jac_step = float(params.get('traj_jac_step', DEFAULTS['traj_jac_step']))

    min_decay = float(params.get('min_step_decay_strength', DEFAULTS['min_step_decay_strength']))
    max_decay = float(params.get('max_step_decay_strength', DEFAULTS['max_step_decay_strength']))

    iter_ratio = np.arange(n_iter, dtype=float) / max(1, n_iter - 1)
    min_step_deg_it = np.maximum(0.0, min_step_deg * (1.0 - min_decay * iter_ratio))
    max_step_deg_it = np.maximum(min_step_deg_it, max_step_deg * (1.0 - max_decay * iter_ratio))

    alpha_it = np.array([alpha_for_iter(i, n_iter, fov_schedule) for i in range(n_iter)], dtype=float)
    ks_it = np.full(n_iter, ks, dtype=float)  # ks is constant in current optimizer implementation.

    return {
        'ks_it': ks_it,
        'alpha_it': alpha_it,
        'base_step_scale': base_step_scale,
        'traj_jac_step': traj_jac_step,
        'min_step_deg_it': min_step_deg_it,
        'max_step_deg_it': max_step_deg_it,
        'fov_schedule': fov_schedule,
    }


def visibility_proxy(positions, dirs, points, ks_it, alpha_it):
    n_iter = positions.shape[0]
    vis = np.zeros(n_iter, dtype=float)
    for it in range(n_iter):
        ca = np.cos(np.deg2rad(alpha_it[it]))
        acc = 0.0
        for p, d in zip(positions[it], dirs[it]):
            rel = normalize(points - p)
            u = rel @ d
            logits = np.clip(ks_it[it] * (u - ca), -60.0, 60.0)
            v = 1.0 / (1.0 + np.exp(-logits))
            acc += float(v.mean())
        vis[it] = acc / len(positions[it])
    return vis


def mean_rot_change_deg(dirs):
    dot = np.sum(dirs[1:] * dirs[:-1], axis=-1)
    dot = np.clip(dot, -1.0, 1.0)
    return np.degrees(np.arccos(dot)).mean(axis=1)


def per_pose_rot_change_deg(dirs):
    dot = np.sum(dirs[1:] * dirs[:-1], axis=-1)
    dot = np.clip(dot, -1.0, 1.0)
    return np.degrees(np.arccos(dot))  # [iter-1, pose]


def pick_probe_iters(n_iter, probes):
    out = []
    for v in probes:
        i = n_iter - 1 if int(v) < 0 else int(v)
        i = max(0, min(n_iter - 1, i))
        out.append(i)
    return sorted(set(out))


def print_probe_table(rows):
    if not rows:
        print('No rows.')
        return
    headers = [
        'iter', 'alpha_deg', 'ks', 'min_step_deg', 'max_step_deg',
        'visibility', 'mean_rotchg_deg'
    ]
    widths = {h: len(h) for h in headers}
    for r in rows:
        for h in headers:
            widths[h] = max(widths[h], len(str(r[h])))

    def fmt_row(d):
        return ' | '.join(str(d[h]).rjust(widths[h]) for h in headers)

    print(fmt_row({h: h for h in headers}))
    print('-+-'.join('-' * widths[h] for h in headers))
    for r in rows:
        print(fmt_row(r))


def parse_trials_jsonl(path: Path):
    if not path.exists():
        return []
    out = []
    with path.open('r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                out.append(json.loads(line))
            except Exception:
                pass
    return out


In [ ]:
BEST_PARAMS_YAML, BEST_FIXED_PARAMS = parse_best_params(TRACE_ROOT)
RUN_ROOT = resolve_run_root(TRACE_ROOT, VIEW, VARIANT, USE_PATH_YAW, RUN_ROOT_OVERRIDE)

if not RUN_ROOT.exists():
    raise FileNotFoundError(f'Run folder does not exist: {RUN_ROOT}')

QF = resolve_quiver_file(RUN_ROOT)
PF = RUN_ROOT / 'trajectory_pointcloud.csv'
EF = resolve_pose_errors_file(RUN_ROOT)

if not PF.exists():
    raise FileNotFoundError(f'Missing point cloud file: {PF}')

raw = parse_quiver_iterations(QF)
pts = load_points(PF, max_points=MAX_POINTS)

n_iter, n_pose, _ = raw.shape
pos = raw[:, :, :3]
dirs = normalize(raw[:, :, 3:6])

params = dict(DEFAULTS)
params.update(BEST_FIXED_PARAMS)
params.update(PARAM_OVERRIDE)

trace = build_iteration_param_trace(n_iter, params)
visibility = visibility_proxy(pos, dirs, pts, trace['ks_it'], trace['alpha_it'])
rotchg = mean_rot_change_deg(dirs)
per_pose_rotchg = per_pose_rot_change_deg(dirs)

te_vals, re_vals = parse_pose_errors(EF)
te_mean = float(np.mean(te_vals)) if te_vals else np.nan
re_mean = float(np.mean(re_vals)) if re_vals else np.nan

RESULT = {
    'view': VIEW,
    'variant': VARIANT,
    'run_root': RUN_ROOT,
    'quivers_file': QF,
    'points_file': PF,
    'pose_errors_file': EF,
    'n_iter': n_iter,
    'n_pose': n_pose,
    'n_points_loaded': int(pts.shape[0]),
    'params': params,
    'trace': trace,
    'pos': pos,
    'dirs': dirs,
    'points': pts,
    'visibility': visibility,
    'mean_rotchg': rotchg,
    'per_pose_rotchg': per_pose_rotchg,
    'te_values': np.asarray(te_vals, dtype=float),
    're_values': np.asarray(re_vals, dtype=float),
    'te_mean': te_mean,
    're_mean': re_mean,
    'te_count': len(te_vals),
    're_count': len(re_vals),
}

print('Selected run:')
print(f"- root: {RUN_ROOT}")
print(f"- quivers: {QF.name}")
print(f"- iter={n_iter}, poses={n_pose}, loaded_points={pts.shape[0]}")
print('')
print('Resolved params (fixed-mode + override):')
print(f"- ks={trace['ks_it'][0]:.6f}")
print(f"- fov_schedule={trace['fov_schedule']}")
print(f"- base_step_scale={trace['base_step_scale']:.6f}")
print(f"- min_step_deg(start/end)={trace['min_step_deg_it'][0]:.6f}/{trace['min_step_deg_it'][-1]:.6f}")
print(f"- max_step_deg(start/end)={trace['max_step_deg_it'][0]:.6f}/{trace['max_step_deg_it'][-1]:.6f}")
print(f"- traj_jac_step={trace['traj_jac_step']:.6f}")
print('')
print('Final metrics:')
print(f"- final visibility={visibility[-1]:.6f}")
print(f"- TE mean={te_mean:.6f}, RE mean={re_mean:.6f} from n={len(te_vals)} rows")

if BEST_PARAMS_YAML is not None:
    print('')
    print(f"Best params source: {BEST_PARAMS_YAML}")


In [ ]:
it = np.arange(RESULT['n_iter'])
trace = RESULT['trace']
te_values = RESULT['te_values']
re_values = RESULT['re_values']

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

axes[0, 0].plot(it, RESULT['visibility'], marker='o', ms=3, color='tab:blue')
axes[0, 0].set_title('Visibility per iteration')
axes[0, 0].set_xlabel('iteration')
axes[0, 0].set_ylabel('visibility (proxy)')

if te_values.size > 0 or re_values.size > 0:
    if te_values.size > 0:
        axes[0, 1].plot(np.arange(te_values.size), te_values, marker='o', ms=3, color='tab:orange', label='TE')
    if re_values.size > 0:
        axes[0, 1].plot(np.arange(re_values.size), re_values, marker='o', ms=3, color='tab:green', label='RE')
    axes[0, 1].legend(fontsize=9)
else:
    axes[0, 1].text(0.5, 0.5, 'No TE/RE rows found', ha='center', va='center')
axes[0, 1].set_title('TE and RE')
axes[0, 1].set_xlabel('row index in pose_errors')
axes[0, 1].set_ylabel('error')

axes[1, 0].plot(np.arange(1, RESULT['n_iter']), RESULT['mean_rotchg'], marker='o', ms=3, color='tab:purple')
axes[1, 0].set_title('Mean rotation change between iterations')
axes[1, 0].set_xlabel('iteration')
axes[1, 0].set_ylabel('deg')

axes[1, 1].plot(it, trace['alpha_it'], marker='o', ms=3, label='alpha_deg', color='tab:red')
axes[1, 1].plot(it, trace['ks_it'], marker='.', ms=2, label='ks', color='tab:brown')
axes[1, 1].plot(it, trace['min_step_deg_it'], ls='--', label='min_step_deg_it', color='tab:gray')
axes[1, 1].plot(it, trace['max_step_deg_it'], ls='--', label='max_step_deg_it', color='tab:cyan')
axes[1, 1].set_title('Parameter trace used in optimization')
axes[1, 1].set_xlabel('iteration')
axes[1, 1].set_ylabel('value')
axes[1, 1].legend(fontsize=8)

plt.tight_layout()
plt.show()


In [ ]:
probe_ids = pick_probe_iters(RESULT['n_iter'], PROBE_ITERS)
rows = []
for i in probe_ids:
    mean_chg = np.nan if i == 0 else float(RESULT['mean_rotchg'][i - 1])
    rows.append({
        'iter': i,
        'alpha_deg': f"{RESULT['trace']['alpha_it'][i]:.3f}",
        'ks': f"{RESULT['trace']['ks_it'][i]:.6f}",
        'min_step_deg': f"{RESULT['trace']['min_step_deg_it'][i]:.6f}",
        'max_step_deg': f"{RESULT['trace']['max_step_deg_it'][i]:.6f}",
        'visibility': f"{RESULT['visibility'][i]:.6f}",
        'mean_rotchg_deg': f"{mean_chg:.6f}" if np.isfinite(mean_chg) else 'n/a',
    })

print('Parameter snapshots at selected iterations:')
print_probe_table(rows)


In [ ]:
heat = RESULT['per_pose_rotchg']  # [iter-1, pose]

plt.figure(figsize=(13, 4))
plt.imshow(
    heat.T,
    aspect='auto',
    origin='lower',
    cmap='magma',
)
plt.colorbar(label='rotation change (deg)')
plt.title('Per-pose rotation change (iteration transition x pose)')
plt.xlabel('iteration transition (i-1 -> i)')
plt.ylabel('pose index')
plt.tight_layout()
plt.show()


In [ ]:
TRIALS_PATH = TRACE_ROOT / 'optuna_trials.jsonl'
records = parse_trials_jsonl(TRIALS_PATH)

if not records:
    print(f'No trials file found: {TRIALS_PATH}')
else:
    tune = [r for r in records if 'number' in r]
    fixed = [r for r in records if r.get('mode') == 'fixed']

    print(f'Trials file: {TRIALS_PATH}')
    print(f'- tune trials: {len(tune)}')
    print(f'- fixed runs : {len(fixed)}')

    if tune:
        tune_sorted = sorted(tune, key=lambda x: x.get('number', -1))
        last = tune_sorted[-min(8, len(tune_sorted)):]

        print('')
        print('Last tune trials (trial, combined_score, ks, base_step_scale, min/max step, schedule):')
        for r in last:
            u = r.get('user_attrs', {}) or {}
            rp = u.get('resolved_params', {}) or {}
            print(
                f"- {r.get('number')}: score={r.get('value'):.6f}, "
                f"ks={rp.get('ks', np.nan):.6f}, "
                f"base={rp.get('base_step_scale', np.nan):.6f}, "
                f"min={rp.get('min_step_deg', np.nan):.4f}, "
                f"max={rp.get('max_step_deg', np.nan):.4f}, "
                f"sched={rp.get('fov_schedule', 'n/a')}"
            )

        xs = []
        ys_score = []
        ys_ks = []
        for r in tune_sorted:
            u = r.get('user_attrs', {}) or {}
            rp = u.get('resolved_params', {}) or {}
            if r.get('number') is None or r.get('value') is None:
                continue
            xs.append(int(r['number']))
            ys_score.append(float(r['value']))
            ys_ks.append(float(rp.get('ks', np.nan)))

        fig, axes = plt.subplots(1, 2, figsize=(13, 4))
        axes[0].plot(xs, ys_score, marker='o', ms=3)
        axes[0].set_title('Tune combined score by trial')
        axes[0].set_xlabel('trial number')
        axes[0].set_ylabel('combined score')

        axes[1].plot(xs, ys_ks, marker='o', ms=3, color='tab:orange')
        axes[1].set_title('Resolved ks by trial')
        axes[1].set_xlabel('trial number')
        axes[1].set_ylabel('ks')

        plt.tight_layout()
        plt.show()

    if fixed:
        print('')
        print('Fixed-mode records:')
        for r in fixed:
            p = r.get('params', {}) or {}
            u = r.get('user_attrs', {}) or {}
            print(
                f"- score={r.get('value', np.nan):.6f}, "
                f"TE={u.get('te', np.nan):.6f}, RE={u.get('re', np.nan):.6f}, "
                f"ks={float(p.get('ks', np.nan)):.6f}, "
                f"sched={p.get('fov_schedule', 'n/a')}"
            )


In [ ]:
if go is None:
    print('Plotly is not installed. Install with: pip install plotly')
else:
    def _equal_scene_ranges(points_xyz, all_pos_xyz, pad_ratio=0.05):
        xyz = np.vstack([points_xyz, all_pos_xyz])
        lo = xyz.min(axis=0)
        hi = xyz.max(axis=0)
        center = 0.5 * (lo + hi)
        span = float(np.max(hi - lo))
        pad = span * pad_ratio if span > 0 else 1.0
        half = 0.5 * span + pad
        return [
            [center[0] - half, center[0] + half],
            [center[1] - half, center[1] + half],
            [center[2] - half, center[2] + half],
            span,
        ]


    def make_quiver_animation(result, point_stride=3, cone_scale=0.04, max_frames=150):
        pos = result['pos']
        dirs = result['dirs']
        pts = result['points']

        if point_stride is not None and point_stride > 1:
            pts = pts[::int(point_stride)]

        n_iter, n_pose, _ = pos.shape

        frame_step = max(1, int(np.ceil(n_iter / max_frames)))
        frame_ids = list(range(0, n_iter, frame_step))
        if frame_ids[-1] != n_iter - 1:
            frame_ids.append(n_iter - 1)

        xr, yr, zr, span = _equal_scene_ranges(pts, pos.reshape(-1, 3))
        vec_scale = max(1e-6, span * float(cone_scale))
        pose_idx = np.arange(n_pose)

        i0 = frame_ids[0]
        fig = go.Figure(
            data=[
                go.Scatter3d(
                    x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
                    mode='markers',
                    marker=dict(size=1.4, color='rgba(110,110,110,0.30)'),
                    name='point cloud',
                    hoverinfo='skip',
                ),
                go.Scatter3d(
                    x=pos[0, :, 0], y=pos[0, :, 1], z=pos[0, :, 2],
                    mode='lines',
                    line=dict(width=3, color='rgba(80,80,80,0.6)', dash='dash'),
                    name='iter 0 path',
                    hoverinfo='skip',
                ),
                go.Scatter3d(
                    x=pos[-1, :, 0], y=pos[-1, :, 1], z=pos[-1, :, 2],
                    mode='lines',
                    line=dict(width=3, color='rgba(0,140,90,0.75)', dash='dot'),
                    name='final path',
                    hoverinfo='skip',
                ),
                go.Scatter3d(
                    x=pos[i0, :, 0], y=pos[i0, :, 1], z=pos[i0, :, 2],
                    mode='lines+markers',
                    line=dict(width=6, color='rgb(33,150,243)'),
                    marker=dict(size=4, color=pose_idx, colorscale='Viridis'),
                    customdata=pose_idx[:, None],
                    hovertemplate='pose=%{customdata[0]}<br>x=%{x:.3f}<br>y=%{y:.3f}<br>z=%{z:.3f}<extra></extra>',
                    name='current path / poses',
                ),
                go.Cone(
                    x=pos[i0, :, 0], y=pos[i0, :, 1], z=pos[i0, :, 2],
                    u=dirs[i0, :, 0] * vec_scale,
                    v=dirs[i0, :, 1] * vec_scale,
                    w=dirs[i0, :, 2] * vec_scale,
                    anchor='tail',
                    sizemode='absolute',
                    sizeref=1.0,
                    colorscale='Turbo',
                    showscale=False,
                    name='view direction cones',
                    hoverinfo='skip',
                ),
            ]
        )

        frames = []
        slider_steps = []
        for it in frame_ids:
            frames.append(
                go.Frame(
                    name=str(it),
                    data=[
                        go.Scatter3d(
                            x=pos[it, :, 0], y=pos[it, :, 1], z=pos[it, :, 2],
                            mode='lines+markers',
                            line=dict(width=6, color='rgb(33,150,243)'),
                            marker=dict(size=4, color=pose_idx, colorscale='Viridis'),
                            customdata=pose_idx[:, None],
                            hovertemplate='pose=%{customdata[0]}<br>x=%{x:.3f}<br>y=%{y:.3f}<br>z=%{z:.3f}<extra></extra>',
                            name='current path / poses',
                        ),
                        go.Cone(
                            x=pos[it, :, 0], y=pos[it, :, 1], z=pos[it, :, 2],
                            u=dirs[it, :, 0] * vec_scale,
                            v=dirs[it, :, 1] * vec_scale,
                            w=dirs[it, :, 2] * vec_scale,
                            anchor='tail',
                            sizemode='absolute',
                            sizeref=1.0,
                            colorscale='Turbo',
                            showscale=False,
                            name='view direction cones',
                            hoverinfo='skip',
                        ),
                    ],
                    traces=[3, 4],
                )
            )
            slider_steps.append(
                {
                    'args': [[str(it)], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate'}],
                    'label': str(it),
                    'method': 'animate',
                }
            )

        fig.frames = frames
        fig.update_layout(
            title=(
                f"{result['view']} ({result['variant']}) | iter 0 -> {n_iter - 1} | "
                f"final visibility={result['visibility'][-1]:.4f}"
            ),
            scene=dict(
                xaxis=dict(range=xr, title='x'),
                yaxis=dict(range=yr, title='y'),
                zaxis=dict(range=zr, title='z'),
                aspectmode='cube',
            ),
            margin=dict(l=0, r=0, t=45, b=0),
            legend=dict(x=0.01, y=0.99),
            uirevision='keep-camera',
            updatemenus=[
                {
                    'type': 'buttons',
                    'showactive': False,
                    'x': 0.0,
                    'y': 1.08,
                    'buttons': [
                        {
                            'label': 'Play',
                            'method': 'animate',
                            'args': [
                                None,
                                {
                                    'fromcurrent': True,
                                    'frame': {'duration': 120, 'redraw': True},
                                    'transition': {'duration': 0},
                                },
                            ],
                        },
                        {
                            'label': 'Pause',
                            'method': 'animate',
                            'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate'}],
                        },
                    ],
                }
            ],
            sliders=[
                {
                    'active': 0,
                    'currentvalue': {'prefix': 'iteration: '},
                    'pad': {'t': 45},
                    'steps': slider_steps,
                }
            ],
        )
        return fig


    fig = make_quiver_animation(
        RESULT,
        point_stride=POINT_STRIDE_3D,
        cone_scale=0.04,
        max_frames=MAX_ANIM_FRAMES,
    )
    try:
        fig.show()
    except Exception as e:
        print(f'Interactive render fallback (non-notebook environment): {e}')
        fig
